In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics



In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("Adult")
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
# create a Spark Session instead of a Spark Context
spark_context =  SparkContext.getOrCreate(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 23:18:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load and parse the data file into an RDD of LabeledPoint.
train_data = MLUtils.loadLibSVMFile(spark_context, '/work/li.baol/data/a9a')
test_data = MLUtils.loadLibSVMFile(spark_context, '/work/li.baol/data/a9a.t')
# Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

In [5]:
train_data = train_data.map(lambda x: x if x.label == 1 else LabeledPoint(0, x.features))
test_data = test_data.map(lambda x: x if x.label == 1 else LabeledPoint(0, x.features))

In [6]:
num_feature = train_data.take(1)[0].features.size
num_feature

123

In [7]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
categorical = {}
for i in range(num_feature):
    categorical[i] = 2 # two types
model = DecisionTree.trainClassifier(train_data, numClasses=2, categoricalFeaturesInfo=categorical,
                                    impurity='gini', maxDepth=8)

22/11/28 23:18:12 WARN BlockManager: Task 6 already completed, not releasing lock for rdd_2_0


In [8]:
# Evaluate model on test instances and compute test error
predictions = model.predict(test_data.map(lambda x: x.features))
predictions

MapPartitionsRDD[47] at mapPartitions at PythonMLLibAPI.scala:1344

In [9]:
predictions.take(15), test_data.map(lambda x: x.label).take(15)

([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0])

In [10]:
predsAndLabels = predictions.zip(test_data.map(lambda lp: lp.label))
# testMSE = predsAndLabels.map(lambda lp: (lp[1] - lp[0]) * (lp[1] - lp[0])).sum() /\
#     float(test_data.count())
metrics = BinaryClassificationMetrics(predsAndLabels)    
print(f'AUC = {metrics.areaUnderROC}')


/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


AUC = 0.7416637654649734
